$\newcommand\reals{\mathbf{R}}$
$\newcommand\cd{\mathcal{D}}$
$\newcommand\ch{\mathcal{H}}$

# 2. Calculating Subgradients

## 2.1
Suppose $f_{1},\ldots,f_{m}:\reals^{d}\to\reals$
are convex functions, and 
\begin{align}
f(x)=\max_{i=1,\ldots,,m}f_{i}(x).
\end{align}
Let $k$ be any index for which $f_{k}(x)=f(x)$, and choose $g\in\partial f_{k}(x)$.
Show that $g\in\partial f(x)$.

#### Proof:
Since $g \in\partial f_k(x)$, we have,

\begin{align}
f_k(x+v) &\geq f_k(x) + g^Tv &&\forall v\in\reals^d
\end{align}

But 
\begin{align}
f(x) &= \max_{i=1,\ldots,,m}f_{i}(x)\\
&\geq f_{i}(x) && \forall i=1,\ldots,,m
\end{align}

Thus $\forall y\in \reals^d$, we have $f(y) \geq f_k(y)$

Now observe that
\begin{align}
f(x+v) &\geq f_k(x+v)\\
&\geq f_k(x) + g^Tv\\
&= f(x) + g^Tv && \text{since $f(x)=f_k(x)$}
\end{align}

Thus $g\in\partial f(x)$.

\newpage

## 2.2
Give a subgradient of $J(w)=\max\left\{ 0,1-yw^{T}x\right\} .$


From 2.1 we have

\begin{align}
\partial J(w) = \begin{cases}
-yx & yw^Tx \leq 1\\
0 & otherwise
\end{cases}
\end{align}

\newpage

# 3. Perceptron

## 3.1
Show that if $\left\{ x\mid w^{T}x=0\right\} $ is a separating hyperplane
for a training set $\cd=\left(\left(x_{1},y_{1}\right),\ldots,(x_{n},y_{n})\right)$,
then the average perceptron loss on $\cd$ is $0$. Thus any separating
hyperplane of $\cd$ is an empirical risk minimizer for perceptron
loss.

#### Proof

Assume $\{x\, \lvert\, w^Tx=0\}$ is a separating hyperplane. Then $y_iw^Tx_i > 0\,\, \forall\,\, i \in \{1, \ldots, n\}$.

Now observe that

\begin{align}
\text{Average perceptron loss} &= \frac{1}{n} \sum_{i=1}^n max\{0, -\hat{y_i}y_i\}\\
&= \frac{1}{n} \sum_{i=1}^n max\{0, -y_iw^Tx_i\}\\
&= \frac{1}{n} \sum_{i=1}^n 0 && \text{since $y_iw^Tx_i > 0$}\\
&= 0
\end{align}

\newpage

## 3.2
Let $\ch$ be the linear hypothesis space consisting of functions
$x\mapsto w^{T}x$. Consider running stochastic subgradient descent
(SSGD) to minimize the empirical risk with the perceptron loss. We'll
use the version of SSGD in which we cycle through the data points
in each epoch. Show that if we use a fixed step size $1$, we terminate
when our training data are separated, and we make the right choice
of subgradient,  then we are exactly doing the Perceptron algorithm.

By 2.1 we have $ \partial l(\hat{y},y) = \begin{cases}
-yx & yw^Tx < 0\\
0 & otherwise
\end{cases}$

With this choice of subgradient and a step size of 1 we can see that the update step is $w^{(k+1)} = \begin{cases}
w^{(k)} + y_ix_i & yw^Tx < 0\\
w^{(k)} & otherwise
\end{cases}$.

Note that this is precisely the update step in the perceptron algorithm.

Now for the terminating conditions.

Note that when the training data is separated, we have $y_iw^Tx_i > 0\,\, \forall\,\, i \in \{1, \ldots, n\}$. In the perceptron algorithm this will result in all_correct = True and termination of that algorithm. Thus the terminating conditions are equivalent.